In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

In [7]:
data = "output.xlsx"

# Load data - filter for SUCCESSFUL saves only
df = pd.read_excel(data, sheet_name="Shots")
df_saves = df[df['saved'] == True].copy()

In [8]:
X = df_saves[['x', 'y', 'velocity']]

# Output targets (all body node coordinates)
body_nodes = [
    'torso_x', 'torso_y', 'head_x', 'head_y',
    'left_shoulder_x', 'left_shoulder_y', 'left_elbow_x', 'left_elbow_y',
    'left_hand_x', 'left_hand_y', 'left_hip_x', 'left_hip_y',
    'left_knee_x', 'left_knee_y', 'left_foot_x', 'left_foot_y',
    'right_shoulder_x', 'right_shoulder_y', 'right_elbow_x', 'right_elbow_y',
    'right_hand_x', 'right_hand_y', 'right_hip_x', 'right_hip_y',
    'right_knee_x', 'right_knee_y', 'right_foot_x', 'right_foot_y'
]

y = df_saves[body_nodes]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Multi-output Random Forest
model = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42)
)
model.fit(X_train, y_train)

# Predict pose for new shot
new_shot = np.array([[13.5, 9.2, 120]])  # x, y, velocity
predicted_pose = model.predict(new_shot)

print("Predicted goalkeeper pose:")
for node, value in zip(body_nodes, predicted_pose[0]):
    print(f"{node}: {value:.2f}")

# Evaluate
from sklearn.metrics import mean_absolute_error, r2_score
y_pred = model.predict(X_test)
print(f"\nMean Absolute Error: {mean_absolute_error(y_test, y_pred):.3f}")
print(f"R² Score: {r2_score(y_test, y_pred):.3f}")

c:\Users\joaod\miniconda3\envs\avi\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\joaod\miniconda3\envs\avi\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\joaod\miniconda3\envs\avi\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\joaod\miniconda3\envs\avi\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\joaod\miniconda3\envs\avi\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegress

Predicted goalkeeper pose:
torso_x: 13.13
torso_y: 13.70
head_x: 12.99
head_y: 16.09
left_shoulder_x: 11.12
left_shoulder_y: 15.20
left_elbow_x: 14.15
left_elbow_y: 12.52
left_hand_x: 12.82
left_hand_y: 11.13
left_hip_x: 11.62
left_hip_y: 10.68
left_knee_x: 7.54
left_knee_y: 6.59
left_foot_x: 11.50
left_foot_y: 4.26
right_shoulder_x: 15.12
right_shoulder_y: 15.20
right_elbow_x: 16.23
right_elbow_y: 12.26
right_hand_x: 13.86
right_hand_y: 10.73
right_hip_x: 14.63
right_hip_y: 10.68
right_knee_x: 11.07
right_knee_y: 5.85
right_foot_x: 14.64
right_foot_y: 3.54

Mean Absolute Error: 0.344
R² Score: 0.921


In [9]:
# Full pipeline for your notebook

# 1. Load and prepare data
df_all = pd.read_excel('output.xlsx')

# 2. Create prediction features
X_all = df_all[['x', 'y', 'velocity']]

# 3. Generate predictions
print("Generating optimal poses for all shots...")
predicted_poses = model.predict(X_all)

# 4. Create new dataframe
df_with_predictions = df_all.copy()

# 5. Replace pose columns
body_node_columns = [
    'torso_x', 'torso_y', 'head_x', 'head_y',
    'left_shoulder_x', 'left_shoulder_y', 'left_elbow_x', 'left_elbow_y',
    'left_hand_x', 'left_hand_y', 'left_hip_x', 'left_hip_y',
    'left_knee_x', 'left_knee_y', 'left_foot_x', 'left_foot_y',
    'right_shoulder_x', 'right_shoulder_y', 'right_elbow_x', 'right_elbow_y',
    'right_hand_x', 'right_hand_y', 'right_hip_x', 'right_hip_y',
    'right_knee_x', 'right_knee_y', 'right_foot_x', 'right_foot_y'
]

for i, col in enumerate(body_node_columns):
    df_with_predictions[col] = predicted_poses[:, i]

# 6. Optionally recalculate derived features
from scipy.spatial.distance import cdist

def recalculate_features(row):
    """Recalculate nearest_node, distance, radius based on new pose"""
    shot_point = np.array([row['x'], row['y']])
    
    # Get all body node coordinates
    nodes = []
    for col in body_node_columns:
        if '_x' in col:
            node_name = col.replace('_x', '')
            node_x = row[col]
            node_y = row[node_name + '_y']
            nodes.append((node_name, node_x, node_y))
    
    # Find nearest node
    min_distance = float('inf')
    nearest = None
    
    for node_name, nx, ny in nodes:
        dist = np.sqrt((shot_point[0] - nx)**2 + (shot_point[1] - ny)**2)
        if dist < min_distance:
            min_distance = dist
            nearest = node_name
    
    return nearest, min_distance

# Apply to all rows
print("Recalculating nearest_node and distance...")
results = df_with_predictions.apply(recalculate_features, axis=1)
df_with_predictions['nearest_node'] = [r[0] for r in results]
df_with_predictions['distance'] = [r[1] for r in results]

# 7. Save
df_with_predictions.to_excel('output_with_optimal_poses.xlsx', index=False)
print(f"\n✅ Created dataframe with {len(df_with_predictions)} shots")
print(f"✅ All body poses replaced with ML-predicted optimal positions")
print(f"✅ Saved to: output_with_optimal_poses.xlsx")

# 8. Show sample comparison
print("\n" + "="*80)
print("SAMPLE COMPARISON - Shot #0")
print("="*80)
print(f"\nShot characteristics:")
print(f"  Position: ({df_all.iloc[0]['x']:.2f}, {df_all.iloc[0]['y']:.2f})")
print(f"  Velocity: {df_all.iloc[0]['velocity']:.2f}")
print(f"  Original saved: {df_all.iloc[0]['saved']}")

print(f"\nLeft hand position:")
print(f"  Original:  ({df_all.iloc[0]['left_hand_x']:.2f}, {df_all.iloc[0]['left_hand_y']:.2f})")
print(f"  Predicted: ({df_with_predictions.iloc[0]['left_hand_x']:.2f}, {df_with_predictions.iloc[0]['left_hand_y']:.2f})")

print(f"\nRight hand position:")
print(f"  Original:  ({df_all.iloc[0]['right_hand_x']:.2f}, {df_all.iloc[0]['right_hand_y']:.2f})")
print(f"  Predicted: ({df_with_predictions.iloc[0]['right_hand_x']:.2f}, {df_with_predictions.iloc[0]['right_hand_y']:.2f})")

Generating optimal poses for all shots...
Recalculating nearest_node and distance...

✅ Created dataframe with 737 shots
✅ All body poses replaced with ML-predicted optimal positions
✅ Saved to: output_with_optimal_poses.xlsx

SAMPLE COMPARISON - Shot #0

Shot characteristics:
  Position: (12.73, 8.38)
  Velocity: 134.47
  Original saved: False

Left hand position:
  Original:  (12.12, 10.49)
  Predicted: (12.15, 10.36)

Right hand position:
  Original:  (13.51, 10.56)
  Predicted: (13.57, 10.37)


In [15]:
# Comprehensive statistical comparison
def generate_comparison_report():
    report = []
    report.append("="*80)
    report.append("GOALKEEPER PERFORMANCE COMPARISON REPORT")
    report.append("="*80)
    
    # Basic stats
    total = len(df_all)
    orig_saves = (df_all['saved'] == True).sum()
    opt_saves = (df_with_predictions['saved'] == True).sum()
    
    report.append(f"\n📊 OVERALL STATISTICS")
    report.append(f"Total shots analyzed: {total}")
    report.append(f"\nOriginal Performance:")
    report.append(f"  • Saves: {orig_saves} ({orig_saves/total*100:.1f}%)")
    report.append(f"  • Goals conceded: {total - orig_saves} ({(total-orig_saves)/total*100:.1f}%)")
    
    report.append(f"\nOptimal ML Performance:")
    report.append(f"  • Saves: {opt_saves} ({opt_saves/total*100:.1f}%)")
    report.append(f"  • Goals conceded: {total - opt_saves} ({(total-opt_saves)/total*100:.1f}%)")
    
    improvement = opt_saves - orig_saves
    report.append(f"\n✨ IMPROVEMENT")
    report.append(f"  • Additional saves: {improvement} (+{improvement/orig_saves*100:.1f}%)")
    report.append(f"  • Percentage point gain: +{improvement/total*100:.1f}pp")
    
    # Distance analysis
    avg_dist_orig = df_all['distance'].mean()
    avg_dist_opt = df_with_predictions['distance'].mean()
    
    report.append(f"\n📏 DISTANCE METRICS")
    report.append(f"  • Original avg distance: {avg_dist_orig:.3f}")
    report.append(f"  • Optimal avg distance: {avg_dist_opt:.3f}")
    report.append(f"  • Improvement: {avg_dist_orig - avg_dist_opt:.3f} ({(avg_dist_orig - avg_dist_opt)/avg_dist_orig*100:.1f}%)")
    
    # By shot characteristics
    report.append(f"\n🎯 BREAKDOWN BY SHOT TYPE")
    
    for velocity_range in [(0, 80), (80, 100), (100, 120), (120, 200)]:
        mask = (df_all['velocity'] >= velocity_range[0]) & (df_all['velocity'] < velocity_range[1])
        if mask.sum() > 0:
            orig_rate = (df_all[mask]['saved'] == 'VERDADEIRO').sum() / mask.sum() * 100
            opt_rate = (df_with_predictions[mask]['saved'] == 'VERDADEIRO').sum() / mask.sum() * 100
            report.append(f"  • {velocity_range[0]}-{velocity_range[1]} km/h: {orig_rate:.1f}% → {opt_rate:.1f}% (+{opt_rate - orig_rate:.1f}pp)")
    
    return "\n".join(report)

print(generate_comparison_report())

# Save report to file
with open('comparison_report.txt', 'w') as f:
    f.write(generate_comparison_report())
print("\n📄 Report saved to 'comparison_report.txt'")

GOALKEEPER PERFORMANCE COMPARISON REPORT

📊 OVERALL STATISTICS
Total shots analyzed: 737

Original Performance:
  • Saves: 247 (33.5%)
  • Goals conceded: 490 (66.5%)

Optimal ML Performance:
  • Saves: 247 (33.5%)
  • Goals conceded: 490 (66.5%)

✨ IMPROVEMENT
  • Additional saves: 0 (+0.0%)
  • Percentage point gain: +0.0pp

📏 DISTANCE METRICS
  • Original avg distance: 1.547
  • Optimal avg distance: 2.078
  • Improvement: -0.531 (-34.3%)

🎯 BREAKDOWN BY SHOT TYPE
  • 0-80 km/h: 0.0% → 0.0% (+0.0pp)
  • 80-100 km/h: 0.0% → 0.0% (+0.0pp)
  • 100-120 km/h: 0.0% → 0.0% (+0.0pp)
  • 120-200 km/h: 0.0% → 0.0% (+0.0pp)

📄 Report saved to 'comparison_report.txt'
